In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import random
import sys
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from topology import Topology
from simulation import Simulation
from simulation import SimulationResult
from packet import PacketSf
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

random.seed(42)  # for now seed is constant


class SimulationFigure():
    def __init__(self, x_axis, plot_names):
        self.x_axis = x_axis
        self.plot_names = plot_names
        self.plot_data = {}
        for plot_name in self.plot_names:
            self.plot_data[plot_name] = []

    def get_plot(self, xlabel, ylabel, ylim_bottom=None, ylim_top=None, xlim_left=None, xlim_right=None):
        plt.figure()
        for plot_name in self.plot_names:
            plt.plot(self.x_axis, self.plot_data[plot_name], label=plot_name)

        if ylim_bottom is not None:
            plt.ylim(bottom=ylim_bottom)
        if ylim_top is not None:
            plt.ylim(top=ylim_top)
        if xlim_left is not None:
            plt.xlim(left=xlim_left)
        if xlim_right is not None:
            plt.xlim(right=xlim_right)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.grid(True)
        plt.tight_layout()

def prediction_pdr(averaging, number_of_gws, packet_rate, packet_size, simulation_duration, traffic_type):
    for radius in [3000, 5000, 7000, 10000]:
        for number_of_nodes in [100, 500, 1000]:
            prediction_rf_pdr_averaging_sum = 0
            prediction_gb_pdr_averaging_sum = 0
            lowest_pdr_averaging_sum = 0
            topology = Topology.create_random_topology(number_of_nodes=number_of_nodes, radius=radius, number_of_gws=number_of_gws, node_traffic_proportions=traffic_type)

            for repeat in range(averaging):
                simulation = Simulation(topology=topology, packet_rate=packet_rate, packet_size=packet_size, simulation_duration=simulation_duration, sf=PacketSf.SF_Random)
                simulation_result = simulation.run()

                X_train, X_test, y_train, y_test = simulation.get_training_data(test_size=0.2)

                # Random Forest Classifier
                RF_classifier = RandomForestClassifier(class_weight='balanced', n_estimators=100)
                RF_classifier.fit(X_train, y_train)
                simulation = Simulation(topology=topology, packet_rate=packet_rate, packet_size=packet_size, simulation_duration=simulation_duration, sf=PacketSf.SF_Smart, sfPredictor=RF_classifier.predict)
                simulation_result = simulation.run()
                prediction_rf_pdr_averaging_sum += simulation_result.pdr

                # Gradient Boosting Classifier
                GB_classifier = GradientBoostingClassifier()
                GB_classifier.fit(X_train, y_train)
                simulation = Simulation(topology=topology, packet_rate=packet_rate, packet_size=packet_size, simulation_duration=simulation_duration, sf=PacketSf.SF_Smart, sfPredictor=GB_classifier.predict)
                simulation_result = simulation.run()
                prediction_gb_pdr_averaging_sum += simulation_result.pdr

                simulation = Simulation(topology=topology, packet_rate=packet_rate, packet_size=packet_size, simulation_duration=simulation_duration, sf=PacketSf.SF_Lowest)
                simulation_result = simulation.run()
                lowest_pdr_averaging_sum += simulation_result.pdr

            print('number_of_nodes={}, radius={}'.format(number_of_nodes, radius))
            print('pdr L={:.1f}, RF={:.1f}, GB={:.1f}'.format(lowest_pdr_averaging_sum / averaging, prediction_rf_pdr_averaging_sum / averaging, prediction_gb_pdr_averaging_sum / averaging))



# All units are SI base units
TOPOLOGY_RADIUS = 3000  # meters
NUMBER_OF_GWS = 1
PRED_TOPOLOGY_RADIUS = 5000  # meters
PRED_NUMBER_OF_GWS = 3
SIMULATION_DURATION = 3600  # seconds
PACKET_RATE = 0.01  # per second
PACKET_SIZE = 60  # bytes, header + payload, 13 + max(51 to 222)
TRAFFIC_TYPE = (1, 0)  # poisson, periodic
AVERAGING = 5
NUMBER_OF_NODES_LIST = range(50, 1001, 50)



prediction_pdr(averaging=AVERAGING,
        number_of_gws=PRED_NUMBER_OF_GWS,
        packet_rate=PACKET_RATE,
        packet_size=PACKET_SIZE,
        simulation_duration=SIMULATION_DURATION,
        traffic_type=TRAFFIC_TYPE)



number_of_nodes=100, radius=3000
pdr L=97.5, RF=97.8, GB=97.3
number_of_nodes=500, radius=3000
pdr L=86.4, RF=87.9, GB=86.4
number_of_nodes=1000, radius=3000
pdr L=71.8, RF=76.0, GB=71.9
number_of_nodes=100, radius=5000
pdr L=97.2, RF=97.6, GB=97.1
number_of_nodes=500, radius=5000
pdr L=84.9, RF=88.7, GB=85.2
number_of_nodes=1000, radius=5000
pdr L=71.6, RF=78.7, GB=71.2
number_of_nodes=100, radius=7000
pdr L=97.5, RF=98.0, GB=97.4
number_of_nodes=500, radius=7000
pdr L=87.4, RF=89.5, GB=87.3
number_of_nodes=1000, radius=7000
pdr L=76.0, RF=80.5, GB=75.9
number_of_nodes=100, radius=10000
pdr L=98.1, RF=97.9, GB=98.0
number_of_nodes=500, radius=10000
pdr L=90.6, RF=90.9, GB=90.9
number_of_nodes=1000, radius=10000
pdr L=81.3, RF=81.6, GB=81.3
